In [59]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [60]:
import warnings
warnings.filterwarnings("ignore", message=".*SQLAlchemy connectable.*")

In [61]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(r"C:\Users\Administrator\LIS\Kenvue\python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [62]:
server = "157.119.230.120,4071"
database = "PowerBI_LISV2"
username = "sa"
password = "Y0m@Sql.!.123"

conn_str = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
UID={username};
PWD={password};
TrustServerCertificate=yes;
"""

In [63]:
conn = pyodbc.connect(conn_str)

In [64]:
query = """ 
WITH CTE AS (
    SELECT
        'GCPLMTManagement' AS SchemaName,
        A.UserName,
        M.EmployeeID,
        M.FullName,
        M.Gender,
        M.Region,
        M.State,
        M.City,
        M.Designation,
        TRY_CONVERT(date, M.LastWorkingDate, 105) AS LastWorkingDate,
        M.Status AS EmpStatus,
        -- A.Status AS AttRawStatus,
        -- A.ApprovalStatus,
        CASE
            WHEN A.Status = 'Present' THEN 'P'
            WHEN A.Status = 'Absent' OR A.Status IS NULL THEN 'A'
            WHEN A.Status = 'Leave' THEN 'L'
            WHEN A.Status IN ('Compensatory Off','Comp Off') THEN 'CO'
            WHEN A.Status IN ('Week Off','WeekOff') THEN 'WO'
            WHEN A.Status IN ('Meeting','Meeting/Training') THEN 'M'
            WHEN A.Status = 'Training' THEN 'T'
            WHEN A.Status IN ('National Holiday','National Hoilday','Holiday','Regional Holiday') THEN 'H'
            WHEN A.Status IN ('Bandh/Strike','Bandh Strike','Bandh / Strike','Market closed','Strike') THEN 'B'
            WHEN A.Status = 'Vacant' THEN 'V'
            WHEN A.Status IN ('Personal','Personal Leave') THEN 'PL'
            WHEN A.Status = 'Casual Leave' THEN 'CL'
            WHEN A.Status = 'Half Day' THEN 'HD'
            ELSE 'Condition not found'
        END AS AttStatus, 
        FORMAT(CONVERT(datetime, A.[Date], 105), 'dd-MM-yyyy') AS Date2,
        RANK() OVER (PARTITION BY A.UserName,A.[Date] ORDER BY A.ISPAttendanceMasterExportId DESC) AS rk
    FROM PowerBI_LISV2.GCPLManagement.ISPAttendanceMasterExport AS A
    LEFT JOIN (SELECT K.* FROM
        (SELECT  *, RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK FROM 
        PowerBI_LISV2.GCPLManagement.UserMaster) AS K
        WHERE K.RNK =1) AS M
        ON A.UserName = M.UserName
    WHERE 
        -- TRY_CONVERT(date, A.[Date], 105) BETWEEN '{date_from}' AND '{date_to}'
        month = 10 and year =2025
        AND M.UserRole NOT IN ('MIS','RSM','SO','Program Manager','Assistant Manager','FO','Office Staff','ASM',
        'Senior Manager','Supervisor','National')
        AND LOWER(A.UserName) NOT LIKE '%test%'
        AND LOWER(A.UserName) NOT LIKE '%sup%'
        AND LOWER(A.FullName) NOT LIKE '%test%'
),
Filtered AS (
    SELECT *
    FROM CTE
    WHERE rk = 1
),
Pivoted AS (
    SELECT *
    FROM Filtered
    )
SELECT *
FROM Pivoted;"""
df = pd.read_sql(query, conn)

In [65]:
df.head()

,SchemaName,UserName,EmployeeID,FullName,Gender,Region,State,City,Designation,LastWorkingDate,EmpStatus,AttStatus,Date2,rk
0,GCPLMTManagement,EMTM007,YM202012259,Deepak Kumar,M,East,Jharkhand,Ranchi,Sharing Promoter,None,True,H,01-10-2025,1
1,GCPLMTManagement,EMTM007,YM202012259,Deepak Kumar,M,East,Jharkhand,Ranchi,Sharing Promoter,None,True,H,02-10-2025,1
2,GCPLMTManagement,EMTM007,YM202012259,Deepak Kumar,M,East,Jharkhand,Ranchi,Sharing Promoter,None,True,P,03-10-2025,1
3,GCPLMTManagement,EMTM007,YM202012259,Deepak Kumar,M,East,Jharkhand,Ranchi,Sharing Promoter,None,True,P,04-10-2025,1
4,GCPLMTManagement,EMTM007,YM202012259,Deepak Kumar,M,East,Jharkhand,Ranchi,Sharing Promoter,None,True,P,05-10-2025,1


In [66]:
uservisitquery = """
WITH CTE AS (
    SELECT
        'GCPLManagement' AS SchemaName,
        (TXT.UserName + '-' + TXT.Visit_Date) AS UserVisitKey,
        TXT.Visit_Date,
        TXT.UserName,
        (TXT.Visit_Date + ' ' + TXT.CHECKIN_TIME)  AS CheckinTime,
        TXT.CHECKIN_TIME,
        TXT.LISStoreCode,
        (TXT.Visit_Date + '-' + TXT.LISStoreCode + '-' + TXT.UserName) AS StoreDateKey,
        TXT.StoreName,
        TXT.Designation,
        (TXT.Visit_Date + ' ' + TXT.CHECKOUT_TIME) AS CheckoutTime,
        TXT.FullName,
        TXT.NoPermissionReason,
        1 AS VisitFlag
        -- DATEPART(DAY, TRY_CONVERT(date, TXT.Visit_Date, 105)) AS DayNum
    FROM (
        SELECT
            FORMAT(CONVERT(datetime, SA.[VisitDate], 105), 'dd-MM-yyyy') AS Visit_Date,
            SA.LISStoreCode,
            SA.StoreName,
            SA.UserName,
            SA.Designation,
            SA.FullName,
            CONVERT(varchar(8), CAST([CheckIn-Time]  AS time), 108) AS CHECKIN_TIME,
            CONVERT(varchar(8), CAST([CheckOut-Time] AS time), 108) AS CHECKOUT_TIME,
            SA.NoPermissionReason
        FROM [PowerBI_LISV2].GCPLManagement.[ISPStoreAttendance_NormalExport] AS SA
        LEFT JOIN (
            SELECT K.*
            FROM (
                SELECT *,
                       RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK
                FROM PowerBI_LISV2.GCPLManagement.UserMaster
            ) AS K
            WHERE K.RNK = 1
        ) AS UM
            ON UM.UserName = SA.UserName
        WHERE 
            -- TRY_CONVERT(date, SA.[VisitDate], 105) BETWEEN '{date_from}' AND '{date_to}'
            [Month] = MONTH(EOMONTH(GETDATE()))
            AND [Year]  = YEAR(EOMONTH(GETDATE()))
			-- True is considered here for all active users 
            -- AND UM.Status = 'True'
            AND SA.UserName NOT LIKE '%Star%'
            AND LOWER(SA.UserName) NOT LIKE '%test%'
    ) AS TXT
),
-- FILTERED is a table created to group by the user visit count and created a unique visit flag
Filtered AS (
    SELECT  
        -- CTE.UserVisitKey,
		CTE.UserName,
        CTE.Visit_Date,
        CTE.CHECKIN_TIME,
        CASE WHEN COUNT(*) >= 1 THEN 1 ELSE 0 END AS VisitFlag
    FROM CTE
    GROUP BY 
	-- CTE.UserVisitKey,
	CTE.UserName,CTE.Visit_Date,CTE.CHECKIN_TIME
),
-- creaing a pivot for day numbers at unique user level
Pivoted AS (
    SELECT *
    FROM Filtered
    )
-- selecting from pivoted table and creating mtd total columns
SELECT *
FROM Pivoted
ORDER BY UserName;"""
df2 = pd.read_sql(uservisitquery, conn)

In [67]:
df2.head()

,UserName,Visit_Date,CHECKIN_TIME,VisitFlag
0,EMTM007,03-10-2025,10:55:28,1
1,EMTM007,04-10-2025,11:01:11,1
2,EMTM007,05-10-2025,07:55:17,1
3,EMTM007,06-10-2025,08:08:37,1
4,EMTM007,08-10-2025,10:44:54,1


In [68]:
final_df= pd.merge(df,df2,how='left', left_on = ['UserName','Date2'], right_on=['UserName','Visit_Date'])

In [72]:
final_df=final_df.fillna(0)
final_df['VisitFlag'] = final_df['VisitFlag'].replace({0: 'No', 1: 'Yes'})
final_df.head()

,SchemaName,UserName,EmployeeID,FullName,Gender,Region,State,City,Designation,LastWorkingDate,EmpStatus,AttStatus,Date2,rk,Visit_Date,CHECKIN_TIME,VisitFlag
0,GCPLMTManagement,EMTM007,YM202012259,Deepak Kumar,M,East,Jharkhand,Ranchi,Sharing Promoter,0,True,H,01-10-2025,1,0,0,No
1,GCPLMTManagement,EMTM007,YM202012259,Deepak Kumar,M,East,Jharkhand,Ranchi,Sharing Promoter,0,True,H,02-10-2025,1,0,0,No
2,GCPLMTManagement,EMTM007,YM202012259,Deepak Kumar,M,East,Jharkhand,Ranchi,Sharing Promoter,0,True,P,03-10-2025,1,03-10-2025,10:55:28,Yes
3,GCPLMTManagement,EMTM007,YM202012259,Deepak Kumar,M,East,Jharkhand,Ranchi,Sharing Promoter,0,True,P,04-10-2025,1,04-10-2025,11:01:11,Yes
4,GCPLMTManagement,EMTM007,YM202012259,Deepak Kumar,M,East,Jharkhand,Ranchi,Sharing Promoter,0,True,P,05-10-2025,1,05-10-2025,07:55:17,Yes


In [73]:
final_df.to_csv('apptrackertest.csv')